### Setup

* h5py - http://www.h5py.org
    * `$ pip install h5py`
* keras - https://keras.io
    * `$ pip install keras`
* TensorFlow - https://www.tensorflow.org
    * `$ pip install tensorflow`
* TensorFrames - https://spark-packages.org/package/databricks/tensorframes 
    * ```$ cd ~/spark/jars 
 $ wget http://dl.bintray.com/spark-packages/maven/databricks/tensorframes/0.8.1-s_2.11/tensorframes-0.8.1-s_2.11.jar```
 
* Spark Deep Learning - https://spark-packages.org/package/databricks/spark-deep-learning
* Flowers dataset - https://www.tensorflow.org/tutorials/image_retraining


In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("chapter-31-Deep-Learning")\
    .getOrCreate()

import os
SPARK_BOOK_DATA_PATH = os.environ['SPARK_BOOK_DATA_PATH']

In [2]:
## from sparkdl import readImages
## not working

In [3]:
img_dir = SPARK_BOOK_DATA_PATH + '/data/deep-learning-images/'
## image_df = readImages(img_dir)

## workaround: https://stackoverflow.com/questions/49643603/spark-deep-learning-import-error
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(img_dir)

In [4]:
# COMMAND ----------

image_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)



In [5]:
image_df.count()

1

In [6]:
image_df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------+
|image                                                                                                           |
+----------------------------------------------------------------------------------------------------------------+
|[file:/home/gong/spark/books/Spark-The-Definitive-Guide/data/deep-learning-images/LICENSE.txt, -1, -1, -1, -1, ]|
+----------------------------------------------------------------------------------------------------------------+



In [7]:
# COMMAND ----------

# from sparkdl import readImages
from pyspark.sql.functions import lit
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))

In [8]:
tulips_df.count()

554

In [11]:
## DONT RUN this

## tulips_df.show(5,False)


#### ERROR

```
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)
```

#### FIX (not working)
https://stackoverflow.com/questions/43490495/how-to-set-notebookapp-iopub-data-rate-limit-and-others-notebookapp-settings-in

In [9]:
daisy_df = ImageSchema.readImages(img_dir + "/daisy").withColumn("label", lit(0))

In [10]:
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])

daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])

In [11]:
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

In [12]:
# COMMAND ----------

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features",
  modelName="InceptionV3")
lr = LogisticRegression(maxIter=1, regParam=0.05, elasticNetParam=0.3,
  labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)

/home/gong/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gong/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gong/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gong/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

Py4JJavaError: An error occurred while calling o78.transform.
: java.lang.NoClassDefFoundError: com/typesafe/scalalogging/slf4j/LazyLogging
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:763)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:468)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:74)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:369)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:349)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:763)
	a

In [13]:
# COMMAND ----------

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select(
  "prediction", "label"))))

NameError: name 'p_model' is not defined

In [ ]:
# COMMAND ----------

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
# a simple UDF to convert the value to a double
def _p1(v):
  return float(v.array[1])
p1 = udf(_p1, DoubleType())
df = tested_df.withColumn("p_1", p1(tested_df.probability))
wrong_df = df.orderBy(expr("abs(p_1 - label)"), ascending=False)
wrong_df.select("filePath", "p_1", "label").limit(10).show()

In [ ]:
# COMMAND ----------

from sparkdl import readImages, DeepImagePredictor
image_df = readImages(img_dir)
predictor = DeepImagePredictor(
  inputCol="image",
  outputCol="predicted_labels",
  modelName="InceptionV3",
  decodePredictions=True,
  topK=10)
predictions_df = predictor.transform(image_df)

In [ ]:
# COMMAND ----------

df = p_model.transform(image_df)

In [ ]:
# COMMAND ----------

from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF
from keras.applications import InceptionV3
registerKerasImageUDF("my_keras_inception_udf", InceptionV3(weights="imagenet"))


# COMMAND ----------